In [1]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re

import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

In [2]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]

metaCollection = stagingDb['Kolominformatie']

lst_soorten_meta = ['Metaal',
 'Artefact',
 'Glas',
 'Stelling',
 'Hout',
 'Spijker',
 'Plaatsing',
 'Keramiek',
 'Put',
 'Vondst',
 'Standplaats',
 'Aardewerk',
 'Munt',
 'Muur',
 'Skelet',
 'Doos',
 'Project',
 'Spoor',
 'Vindplaats',
 'Leer',
 'Steen',
 'Bot',
 'Vlak',
 'Kleipijp']
df_soorten_meta = pd.DataFrame(lst_soorten_meta).rename(columns={0: 'Soort'}) 

In [3]:
grp_aggr = [{'$group': {'_id': {'project': "$brondata.project", 'soort': "$soort"},'teller': {"$sum": 1}}}
           ,{'$replaceRoot': {'newRoot': {'project': "$_id.project", 'soort': "$_id.soort", 'teller': { '$ifNull': ["$teller", 0]}}}}]
grp_aggr_artef = [{"$match": {"soort": 'Artefact'}}
                 ,{'$group': {'_id': {'project': "$brondata.project", 'soort': "$artefactsoort"},'teller': {"$sum": 1}}}
                 ,{'$replaceRoot': {'newRoot': {'project': "$_id.project", 'soort': "$_id.soort", 'teller': { '$ifNull': ["$teller", 0]}}}}]

def AggregateData(df, fase): 
    df['teller'] = pd.to_numeric(df['teller'])
    df = df.groupby('soort').agg({'project':lambda x: list(x), 'teller':lambda x: sum(x)})
    df['Count'] = df.apply(lambda x: len(x.project), axis=1)
    df['Stage'] = fase
    df = df.sort_values('teller', ascending=False).reset_index()
    df = df.rename(columns={'soort': 'Soort', 'project': 'Projecten', 'teller': 'Aantal_Records', 'Count': 'Aantal_projecten'}).sort_index()
    return df

def getData(collection, soort): 
    df1 = pd.DataFrame(list(collection.aggregate(grp_aggr_artef)))
    df2 = pd.DataFrame(list(collection.aggregate(grp_aggr)))

    return AggregateData(df1.append(df2[df2.soort != 'Artefact']), soort)

In [4]:
df_singlestore = getData(analyseCol, 'SingleStore')
df_singlestore.head(5)

,Soort,Projecten,Aantal_Records,Aantal_projecten,Stage
0,Standplaats,[MAGAZIJN],8407,1,SingleStore
1,Aardewerk,"[DC97, DC05, DC22, DC72, DB34, DC36, DC112, DC...",7138,9,SingleStore
2,Plaatsing,[MAGAZIJN],5634,1,SingleStore
3,Foto,[nan],3227,1,SingleStore
4,Vondst,"[DC20, DC179, DB34, DC18, DC24_STADSKANTOOR, D...",3222,11,SingleStore


In [5]:
df_singlestoreclean = getData(analyseColClean, 'SingleStoreClean')
df_singlestoreclean.head(5)

,Soort,Projecten,Aantal_Records,Aantal_projecten,Stage
0,Standplaats,[MAGAZIJN],8407,1,SingleStoreClean
1,Aardewerk,"[DC20, DC112, DC21, DC36, DB34, DC22, DC72, DC...",7138,9,SingleStoreClean
2,Plaatsing,[MAGAZIJN],5634,1,SingleStoreClean
3,Vondst,"[DC9, DC36, DC5, DC21, DC93, DC11, DC24_STADSK...",3318,22,SingleStoreClean
4,Foto,[nan],3227,1,SingleStoreClean


In [6]:
lst_tables = ['Def_Project', 'Def_Put', 'Def_Vondst', 'Def_Spoor', 'Def_Stelling', 'Def_Doos', 'Def_Standplaats', 'Def_Plaatsing', 'Def_Vlak', 'Def_Vindplaats', 'Def_Artefact'] #['Def_Project', 'Def_Vondst', 'Def_Stelling', 'Def_Plaatsing', 'Def_Vindplaats', 'Def_Artefact']
regexProject = re.compile(r'\'project\': \'(.*?)\'') # regex to replace Object

def getProject(brondata):    
    #print(brondata)
    if brondata is None or brondata != "":
        return regexProject.search(brondata).group(1)
    else: 
        return "" 

df_tables_projects = pd.DataFrame()
    
engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
with engine.connect() as connection:
    for table in lst_tables:     
        df = pd.read_sql_query('SELECT brondata from "' + table + '"', connection)
        
        df_out = pd.DataFrame()
        df_out['project'] = df.apply(lambda x: getProject(x['brondata']), axis=1)
        df_out['soort'] = table[4:]
        df_tables_projects = pd.concat([df_tables_projects, pd.DataFrame(df_out).groupby(['project', 'soort']).size().reset_index(name='teller')])

with engine.connect() as connection:
    df = pd.read_sql_query('SELECT brondata, artefactsoort from "Def_Artefact"', connection)

    df_out = pd.DataFrame()
    df_out['project'] = df.apply(lambda x: getProject(x['brondata']), axis=1)
    df_out['soort'] = df['artefactsoort']
    df_tables_projects = pd.concat([df_tables_projects, pd.DataFrame(df_out).groupby(['project', 'soort']).size().reset_index(name='teller')])

        
df_doelsysteem = AggregateData(df_tables_projects, 'Doelsysteem')
df_doelsysteem.head(5)

,Soort,Projecten,Aantal_Records,Aantal_projecten,Stage
0,Artefact,"[DB08, DB34, DC05, DC09, DC11, DC112, DC13, DC...",11165,20,Doelsysteem
1,Standplaats,[MAGAZIJN],8407,1,Doelsysteem
2,Aardewerk,"[DB34, DC05, DC112, DC20, DC21, DC22, DC36, DC...",7138,9,Doelsysteem
3,Plaatsing,[MAGAZIJN],5634,1,Doelsysteem
4,Vondst,"[DB34, DB8, DC11, DC112, DC13, DC14, DC154, DC...",3318,22,Doelsysteem


## Analyse van alle gegevens

Hieronder volgt een overzicht van alle gegevens die in de vier fases beschikbaar zijn. Zo kunnen fouten in de conversie worden opgespoord. 

In [7]:


def getOverview(OnWhat): 
    lst_countColumns = [OnWhat + '_singlestore', OnWhat + '_singlestoreclean', OnWhat + '_doelsysteem']
    
    df = df_singlestore[['Soort', 'Projecten', OnWhat]].merge(df_singlestoreclean[['Soort', OnWhat]], on=['Soort'], how='outer', suffixes=("_singlestore", "_singlestoreclean"))
    df = df.merge(df_doelsysteem[['Soort', OnWhat]], on=['Soort'], how='outer', suffixes=("_singlestoreclean", "_doelsysteem"))

    df = df.sort_values(OnWhat+ '_singlestore', ascending=False) 
    df.rename(columns={'Aantal_Records': 'Aantal_Records_doelsysteem'}, inplace=True)
    df = df.merge(df_soorten_meta, on=['Soort'], how='outer')
    df[lst_countColumns] = df[lst_countColumns].fillna(0)
    df[lst_countColumns] = df[lst_countColumns].astype(int, errors='ignore')

    df = df.style.bar(subset=lst_countColumns, color='#5fba7d', vmax=20000)
    
    return df

getOverview('Aantal_Records')

,Soort,Projecten,Aantal_Records_singlestore,Aantal_Records_singlestoreclean,Aantal_Records_doelsysteem
0,Standplaats,['MAGAZIJN'],8407,8407,8407
1,Aardewerk,"['DC97', 'DC05', 'DC22', 'DC72', 'DB34', 'DC36', 'DC112', 'DC20', 'DC21']",7138,7138,7138
2,Plaatsing,['MAGAZIJN'],5634,5634,5634
3,Foto,[nan],3227,3227,0
4,Vondst,"['DC20', 'DC179', 'DB34', 'DC18', 'DC24_STADSKANTOOR', 'DC11', 'DC154', 'DC22', 'DC112', 'DC93', 'DC97']",3222,3318,3318
5,Doos,['MAGAZIJN'],2773,2804,2804
6,Spoor,"['DC24_STADSKANTOOR', 'DC154', 'DC93', 'DC112', 'DC97', 'DB34', 'DC179']",1281,1666,1666
7,Project,['DELF-IT'],840,840,840
8,Hout,"['DB34', 'DC97', 'DC05', 'DC11', 'DC21', 'DC09', 'DC20', 'DC36', 'DC23', 'DC22', 'DC24_STADSKANTOOR', 'DC154', 'DC93', 'DC179', 'DC18', 'DC13', 'DC112']",783,783,783
9,Kleipijp,"['DC154', 'DC24_STADSKANTOOR', 'DC18', 'DB08', 'DC112', 'DC36', 'DC93', 'DC23', 'DC21', 'DC20', 'DC14']",553,553,553


## Detailanalyse SingleStore naar Doelsysteem

In [8]:
df = df_singlestore.merge(df_doelsysteem, on=['Soort', 'Aantal_Records', 'Aantal_projecten'], how='outer', suffixes=("_singlestore", "_doelsysteem"))
df[df.Stage_singlestore.isnull() | df.Stage_doelsysteem.isnull() ]

,Soort,Projecten_singlestore,Aantal_Records,Aantal_projecten,Stage_singlestore,Projecten_doelsysteem,Stage_doelsysteem
3,Foto,[nan],3227,1,SingleStore,NaN,NaN
4,Vondst,"[DC20, DC179, DB34, DC18, DC24_STADSKANTOOR, D...",3222,11,SingleStore,NaN,NaN
5,Doos,[MAGAZIJN],2773,1,SingleStore,NaN,NaN
6,Spoor,"[DC24_STADSKANTOOR, DC154, DC93, DC112, DC97, ...",1281,7,SingleStore,NaN,NaN
19,Vlak,"[DC179, DC24_STADSKANTOOR, DC154]",34,3,SingleStore,NaN,NaN
22,Put,"[DC24_STADSKANTOOR, DC154, DC179]",13,3,SingleStore,NaN,NaN
23,Artefact,NaN,11165,20,NaN,"[DB08, DB34, DC05, DC09, DC11, DC112, DC13, DC...",Doelsysteem
24,Vondst,NaN,3318,22,NaN,"[DB34, DB8, DC11, DC112, DC13, DC14, DC154, DC...",Doelsysteem
25,Doos,NaN,2804,10,NaN,"[DB34, DC11, DC112, DC154, DC18, DC21, DC5, DC...",Doelsysteem
26,Spoor,NaN,1666,7,NaN,"[DB34, DC112, DC154, DC179, DC24_STADSKANTOOR,...",Doelsysteem
